# My Deploy Steps For KPERM Project

## Create VPS and user
1. Get DigitalOcean droplet with Ubuntu installed.
2. Create a new user in sudo group
    ```
    adduser --gecos "" web
    usermod -aG sudo web
    su web
    ```

## Make ssh login with ssh-keys
3. Make password-less login via ssh from your local cpmputer to server:
    1. If you don't already have ssh key in ~/.ssh at you local computer you must generate then using command ```ssh-keygen```.
    2. Print you local public key using ```cat ~/.ssh/id_rsa.pub```.
    3. Copy your key to clipboard.
    4. Paste it on your server using ```echo <you_public_key> >> ~/.ssh/authorized_keys```. If nessesary make dir ~/.ssh.
    5. Give permission to ```chmod 600 ~/.ssh/authorized_keys```.
    6. Now you can accsess to server without password using ```ssh web@<ip>```.

## Server Security

### Disable root logins via ssh
1. Set ```PermitRootLogin no``` in ```/etc/ssh/sshd_config```.
2. Restart sshd using ```sudo service ssh restart```.

### Install Firewall And Allow only ssh and http/s
```
sudo apt-get install -y ufw
sudo ufw allow ssh
sudo ufw allow http
sudo ufw allow 443/tcp
sudo ufw --force enable
sudo ufw status
```

## Deploing app

### Installing nginx, tools and app

1. For front web-server use **nginx**.
1. As Python WSGI HTTP Server for UNIX use **Gunicorn**.
1. Process monitor and restart - **supervisor**.


```
sudo apt-get -y update
sudo apt-get -y install python3-venv python3-pip
sudo apt-get -y install supervisor nginx git
```

```
git clone https://github.com/<your_repo>
cd app
git checkout ver1.0

python3 -m venv venv
. venv/bin/activate
pip install -r requirements.txt

pip install gunicorn

echo "export FLASK_APP=app.py" >> ~/.profile
```

### Setting Another App params
1. .env

### Gunicorn

1. To start gunicorn with 4 workers use command:
```/home/web/golden_crown_rates_monitor/venv/bin/gunicorn -b localhost:8000 -w 4 golden_crown_rates_monitor:app```
1. How many workers you should use? [Doc](https://docs.gunicorn.org/en/stable/design.html#how-many-workers) says best number is (2*$cores_numbers+1)

### supervisor

#### Config supervisor
Edit conf file at ```nano \etc\supervisor\conf.d\golden_crown_rates_monitor.conf```

```
[program:golden_crown_rates_monitor]
command=/home/web/golden_crown_rates_monitor/venv/bin/gunicorn -b localhost:8000 -w 4 golden_crown_rates_monitor:app
directory=/home/web/golden_crown_rates_monitor
user=web
autostart=true
autorestart=true
stopasgroup=true
killasgroup=true
```

#### Restart supervisor

```sudo supervisorctl reload```

### List of users connected to server
1. Use ```w``` command.
1. Get only ssh connections using ```netstat -tnpa | grep 'ESTABLISHED.*sshd'```.


### NGINX

#### General overview

1. ```/etc/nginx/sites-avaliable/*``` - conf files for all your sites.
1. ```/etc/nginx/sites-enable/*``` - conf files for ENABLED sites. Usually here is stored semlink to sites-avaliable. This way you can quikly manage you sites.
1. ```sudo service nginx start``` starts server.
1. ```sudo service nginx stop``` stops server.
1. ```sudo service nginx restarts``` restarts server.
1. ```sudo service nginx reload``` reloads nginx config without stopping server.
1. ```sudo service nginx status``` shows nginx status.

#### NGINX config

For HTTP:80 proxing edit ```sudo nano /etc/nginx/site-enabled/golden_crown_rates_monitor```.

```
server {
    # прослушивание порта 80 (http)
    listen 80;
    
    # server_name _;
    # Просто _ приведет к ошибкам установки SSL-сертификатов certbot-ом, поэтому явно пишу имя сервера
    server_name korona.kislitsyn.me;
    location / {
        # перенаправлять любые запросы на один и тот же URL-адрес, как на https
        proxy_pass      http://localhost:8000;

    }
}

```


### Let's Encrypt SSL certificates

Read more in [DigitalOcean docs](https://www.digitalocean.com/community/tutorials/how-to-secure-nginx-with-let-s-encrypt-on-ubuntu-20-04)

1. Install certbot and nginx plugin for auto add ssl to nginx.
```
sudo apt install certbot python3-certbot-nginx
````
2. Obtaining an SSL Certificate
```
sudo certbot --nginx -d korona.kislitsyn.me -d www.korona.kislitsyn.me
```
3. If you had errors you have to install certs manually
```
sudo apt install certbot python3-certbot-nginx
```
4. Verifying Certbot Auto-Renewal
```
sudo systemctl status certbot.timer
```
5. **Test** renewal process
```
sudo certbot renew --dry-run
```

## Update App
```
(venv) $ git pull                              # Скачать новую версию
(venv) $ sudo supervisorctl stop microblog     # Остановка текущего сервера
(venv) $ flask db upgrade                      # Обновление базы данных
(venv) $ flask translate compile               # Обновление переводов
(venv) $ sudo supervisorctl start microblog    # Запуск нового сервера
```